In [5]:
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
from sklearn.datasets import make_circles
%pylab inline
import warnings
warnings.filterwarnings('ignore')
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from StringIO import StringIO
import pydot_ng
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

Populating the interactive namespace from numpy and matplotlib


# Question 3:

In [6]:
df = pd.read_table('E:\DataAnalytics_programming\Class Notes\sample exam\Santa.txt', header=None)

In [7]:
import re
dict_alltweets = [[] for x in xrange(0,df.shape[0])] # creating a list of lists with number of tweets
for j in range(0,df.shape[0]):                       # for every tweet
    df_sp =df.ix[j,0].split()                        # split the words by spaces

    temp_list =[]
    list_new=[]
    for i in range(0,len(df_sp)):                    # for every word in the list
        if len(df_sp)>=i+3:                          # if the combination of 3 exists
            temp_list  = df_sp[i:i+3]                # slice the three words from the list of words
            list_new.append(" ".join(temp_list))     # merge them into a single phrase
            list_new=list(Series(list_new).unique()) # Take out unique phrases and save as list
    dict_alltweets[j] = list_new                     #Save list of phrases in the list

In [8]:
for  i  in range(0,len(dict_alltweets)):             # for every tweet
    list1 = dict_alltweets[i]                        # initialize list1
    for j in range(0,len(dict_alltweets)):           # for every tweet
        if j > i:                                    # if tweet j is after tweet i 
            list2 = dict_alltweets[j]                # intialize list2
            matches=0                                # initialize matches
            n1= len(list1)                           # Find lengths of two lists (number of phrases)
            n2=len(list2)                            # iterate over two lists and find common matches of phrases
            for u in range(0,len(list1)):
                for v in range(0,len(list2)):
                    if list1[u]==list2[v] and list1!= list2: # len(set(list1) & set(list2))
                        matches+=1 
            if (float(matches/np.sqrt(n1*n2))>0.5):  # if the cosine comes out to be > 0.5, print else do nothing
                print "tweet "+ str(j+1)+' is a near-duplicate of a previously seen tweet from the file of tweet '+str(i+1)

tweet 3 is a near-duplicate of a previously seen tweet from the file of tweet 1
tweet 4 is a near-duplicate of a previously seen tweet from the file of tweet 2
tweet 5 is a near-duplicate of a previously seen tweet from the file of tweet 2
tweet 5 is a near-duplicate of a previously seen tweet from the file of tweet 4


In [186]:
print len(set(list1) & set(list2))
print set(list1) & set(list2)

11
set(['Santa Claus Is', 'Claus Is Coming', 'Rt si te', 'si te gusta', 'gusta Santa Claus', 'Is Coming To', 'Town #MTVHottest Justin', 'te gusta Santa', '#MTVHottest Justin Bieber', 'Coming To Town', 'To Town #MTVHottest'])


# Question 4:

In [228]:
labels = pd.read_table('E:\DataAnalytics_programming\Class Notes\sample exam\labels.txt', header=None,names=['rater','url','class'])
gold = pd.read_table('E:\DataAnalytics_programming\Class Notes\sample exam\gold.txt', header=None,names=['url','class'])
gold1 = gold 
#appending rows
gold1.append(pd.DataFrame(gold['class']))
#apepnding columns 
gold1['class2'] = gold['class']

,url,class,class2
0,http://1024kbytes.com,broken,broken
1,http://18x.fr,yes,yes
2,http://1ting.com,yes,yes
3,http://3dmgame.com,yes,yes
4,http://3xupdate.com,yes,yes
5,http://6ano.org,broken,broken
6,http://7-win.ru,yes,yes
7,http://7dl.biz,yes,yes
8,http://7elm3aber.com,yes,yes


In [191]:
merged = pd.merge(labels, gold, on='url',how='inner',suffixes=['labels','gold']) 
New_Data = DataFrame(data=merged.groupby(by='rater')['classgold'].count().sort_values(ascending=False))
New_Data.rename(columns={'classgold' : 'count'}, inplace=True)

# DataFrame.merge(Series Name) : used to append column to a Dataset
# merged['match'] = np.where(merged['classlabels']==merged['classgold'],1,0)  # create flag for a match or not
merged.ix[merged['classlabels']== merged['classgold'],'match']=1
merged.ix[merged['classlabels']!= merged['classgold'],'match']=0

New_Data['match']  = merged.groupby(by='rater')['match'].sum().fillna(0)      # find sum of matches for every rater
New_Data['rater']  = New_Data.index.values
New_Data['believability']  = (New_Data['match']+1.0)/(New_Data['count']+2.0)  
New_Data['believability_ratio']  = (New_Data['believability'])/(1.0- New_Data['believability'])

new_merged  = pd.merge(merged, New_Data, on = 'rater',how='left')            # To have the urls also

In [222]:
merged['match'] = np.where(merged['classlabels']==merged['classgold'],1,0)
print merged[:3]

            rater            url classlabels classgold  match
0         atiqnur  http://18x.fr         yes       yes      1
1           maria  http://18x.fr         yes       yes      1
2  a3kvu2c809dok5  http://18x.fr          no       yes      0


In [192]:
Url_Data= DataFrame(new_merged.groupby(by=['url','classlabels'])['believability_ratio'].prod().fillna(0).unstack('classlabels'))

In [193]:
(Url_Data.T.idxmax())

url
http://1024kbytes.com       yes
http://18x.fr               yes
http://1ting.com            yes
http://3dmgame.com          yes
http://3xupdate.com         yes
http://6ano.org          broken
http://7-win.ru             yes
http://7dl.biz              yes
http://7elm3aber.com        yes
dtype: object

In [184]:
New_Data

,count,match,rater,believability,believability_ratio
rater,,,,,
a3kvu2c809dok5,18,13.0,a3kvu2c809dok5,0.700000,2.333333
apul86331somr,7,0.0,apul86331somr,0.111111,0.125000
a2ufd1i8zo1v4g,7,2.0,a2ufd1i8zo1v4g,0.333333,0.500000
atiqnur,5,2.0,atiqnur,0.428571,0.750000
a2gng3wn85say6,5,2.0,a2gng3wn85say6,0.428571,0.750000
maria,4,3.0,maria,0.666667,2.000000
a182jw8u6po60u,2,0.0,a182jw8u6po60u,0.250000,0.333333
a3ux2fxa3nmjcs,1,0.0,a3ux2fxa3nmjcs,0.333333,0.500000
a3rlcgrxa34gc0,1,0.0,a3rlcgrxa34gc0,0.333333,0.500000


# Question 5 

In [13]:
cars = pd.read_csv('E:\DataAnalytics_programming\Class Notes\sample exam\cars.csv')
cars.ix[cars['class']=='unacc','target'] ="Not OK"
cars.ix[cars['class']!='unacc','target'] ="OK"
cars[:3]

,buying,maint,doors,persons,lug_boot,safety,class,target
0,vhigh,vhigh,2,2,small,low,unacc,Not OK
1,vhigh,vhigh,2,2,small,med,unacc,Not OK
2,vhigh,vhigh,2,2,small,high,unacc,Not OK


In [14]:
cars.columns.values

array(['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety',
       'class', 'target'], dtype=object)

In [232]:
def entropy_func(df,column_name,yes):                      # 'Yes' is to print out the Predict OK/Not OK for min_var
    unique_vals = df[column_name].unique()
    Entropy = 0.0
    for i in unique_vals:
        P= float(df[((df['target']=="Not OK") & (df[column_name]==i))].shape[0]) # For a particular column's value, how many P and N
        N= float(df[((df['target']=="OK") & (df[column_name]==i))].shape[0])
        if P==0  or N==0:                                  # Did this to avoid Nans in Entropy
            Entropy +=  0.0
        else:
            Entropy += (-1.0) * ((P* np.log2(P/(P+N))+ N* np.log2(N/(P+N)))) # from Decision Tree presentation in class
    
        if P==0 and yes == 1: 
            print column_name, i , "Predict Not OK"
        elif N==0 and yes == 1: 
            print column_name, i , "Predict OK"    
    
    return Entropy

In [233]:
def lowest_entropy(df,var_list):
    min_ent=99999999999999999999999.0
    for i in var_list:
        if ((min_ent) > entropy_func(df,i,0)): # find the variable with minimum entropy
            min_ent = entropy_func(df,i,0)
            min_var = i 
        else: 
            min_ent = min_ent
            min_var = min_var
    print min_var, min_ent 
    print entropy_func(df,min_var,1)

In [234]:
lowest_entropy(cars,['maint', 'buying', 'doors', 'persons', 'lug_boot', 'safety']) # All variables except target initially

safety 1127.31060077
safety low Predict OK
1127.31060077


In [236]:
cars['buying'].unique()
cars['safety'].unique()

array(['low', 'med', 'high'], dtype=object)

In [198]:
# child nodes after first split
cars_high = cars[cars['safety']=='high']
cars_med = cars[cars['safety']=='med']
cars_low = cars[cars['safety']=='low']

In [237]:
# run for each child node and remaining variables to find the next best split

print 'cars_high:\n',lowest_entropy(cars_high, ['maint', 'doors', 'persons', 'lug_boot', 'buying'])
print 'cars_med:\n',lowest_entropy(cars_med, ['maint', 'doors', 'persons', 'lug_boot', 'buying'])
print 'cars_low:\n',lowest_entropy(cars_low, ['maint', 'doors', 'persons', 'lug_boot', 'buying'])


cars_high:
persons 274.876992842
persons 2 Predict OK
274.876992842
None
cars_med:
persons 283.816053574
persons 2 Predict OK
283.816053574
None
cars_low:
persons 278.539597379
persons 2 Predict OK
278.539597379
None
